In [1]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_molmuv/'
dataset = GraphPropPredDataset(name = 'ogbg-molmuv')


split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

Using backend: pytorch


Downloaded 0.01 GB: 100%|██████████| 7/7 [00:02<00:00,  2.92it/s]


Extracting dataset/muv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 93087/93087 [00:01<00:00, 72316.26it/s]


Saving...


In [2]:
print(dataset[0][0].keys())

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])


In [3]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [4]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    labels.append(label)
    num_nodes += g["num_nodes"]

100%|██████████| 93087/93087 [00:02<00:00, 46477.57it/s]


In [5]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [9]:
_node_list = sparse.csr_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [10]:
data = {
    "node_feats": _node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
np.savez_compressed("ogbg-molmuv.npz", **data)

In [11]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
np.savez_compressed("ogbg-molmuv_task.npz",**task_data)